# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
import pandas as pd

airbnb_last_review = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .csv("/FileStore/Examen/Caso3/airbnb_last_review.tsv")
airbnb_room_type = spark.read \
    .format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .load("/FileStore/Examen/Caso3/airbnb_room_type.xlsx")
airbnb_price = spark.read.option("header", "true").csv("/FileStore/Examen/Caso3/airbnb_price.csv")

# merge
airbnb_total = airbnb_price.join(airbnb_room_type, on="listing_id", how="inner")
airbnb_total = airbnb_total.join(airbnb_last_review, on="listing_id", how="inner")

# limpieza
from pyspark.sql.functions import col, to_date, lower, split

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
    # fecha en formato date
airbnb_total = airbnb_total.withColumn(
    "last_review", 
    to_date(col("last_review"), "MMM dd yyyy")
)
    # minusculas el room_type 
airbnb_total = airbnb_total.withColumn("room_type", lower(col("room_type")))

    # eliminar dollars del price
airbnb_total = airbnb_total.withColumn("price", split(col("price"), " ")[0])


airbnb_total.display()



listing_id,price,nbhood_full,description,room_type,host_name,last_review
2595,225,"Manhattan, Midtown",Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
3831,89,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
5099,200,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
5178,79,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
5238,150,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09
5295,135,"Manhattan, Upper West Side",Beautiful 1br on Upper West Side,entire home/apt,Lena,2019-06-22
5441,85,"Manhattan, Hell's Kitchen",Central Manhattan/near Broadway,private room,Kate,2019-06-23
5803,89,"Brooklyn, South Slope","Lovely Room 1, Garden, Best Area, Legal rental",private room,Laurie,2019-06-24
6021,85,"Manhattan, Upper West Side",Wonderful Guest Bedroom in Manhattan for SINGLES,private room,Claudio,2019-07-05
6848,140,"Brooklyn, Williamsburg",Only 2 stops to Manhattan studio,entire home/apt,Allen & Irina,2019-06-29


### ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.


In [0]:
from pyspark.sql.functions import min, max
first_reviewed = airbnb_total.agg(min("last_review")).collect()[0][0]
last_reviewed = airbnb_total.agg(max("last_review")).collect()[0][0]

print(f"Primera reseña: {first_reviewed}")
print(f"Última reseña: {last_reviewed}")

Primera reseña: 2019-01-01
Última reseña: 2019-07-09


### ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [0]:
nb_private_rooms = airbnb_total.filter(airbnb_total.room_type.contains("private")).count()
print(f"privadas: {nb_private_rooms}")

privadas: 11356



###¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [0]:
from pyspark.sql.functions import avg
avg_price = round(airbnb_total.select(avg("price")).collect()[0][0],2)
print(f"precio promedio: {avg_price}")

precio promedio: 141.78



###Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType

# Esquema del df
schema = StructType([
    StructField("first_reviewed", DateType(), True),
    StructField("last_reviewed", DateType(), True),
    StructField("nb_private_rooms", IntegerType(), True),
    StructField("avg_price", FloatType(), True)
])
# datos para el df
datos = [(first_reviewed, last_reviewed, nb_private_rooms, avg_price)]

# Creacion del df
review_dates = spark.createDataFrame(datos, schema)

review_dates.display()


first_reviewed,last_reviewed,nb_private_rooms,avg_price
2019-01-01,2019-07-09,11356,141.78
